In [1]:
import ee
import geopandas
import datetime
import numpy as np
import pandas as pd
import folium
from ast import literal_eval
from shapely import wkt
from shapely.geometry import box, Point, LineString, Polygon, MultiLineString
from shapely.ops import split
import math
import warnings
import os
from functools import reduce

In [2]:
# ee.Initialize()

#### Run `Sample_TerraClimate_by_Country.py`

* `Global_Samples_Balanced_n100_m250_s210_attempt1.csv`
    * The same as `Global_samples_Balanced_100.csv`
    * Pulled every country except for RUS
    * The output data for every country except for RUS is saved in the `/data/remote/TerraClimate_samples_n100_m250_s210_attempt1_noRUS` folder
    * RUS data was pulled using `Global_Samples_Balanced_n100_m200_s210.csv`
    * The output data for RUS is saved in the `/data/remote/TerraClimate_samples_n100_m200_s210_RUS` folder
    * The aggregate data is saved in `/data/Sample_TerraClimate_2018_n100_m250_s210_attempt1_noRUS.csv` and `Sample_TerraClimate_2018_n100_m200_s210_RUS.csv`
* `Global_Samples_Balanced_n100_m200_s211.csv`
    * The output data is saved in the `/data/remote/TerraClimate_samples_n100_m200_s211` folder
    * The aggregate data is saved in `/data/Sample_TerraClimate_2018_n100_m200_s211.csv`
* `Global_Samples_Balanced_n200_m50_s213.csv`
    * This only downloaded sampled irrigated land data
    * This was downloaded by Adam
    * COL and MMR are incomplete in this download
    * The output data is saved in the `/data/remote/TerraClimate_samples_n200_m50_s213` folder
    * The aggregate data is saved in `/data/Sample_TerraClimate_2018_n200_m50_s213.csv`
    * Merged into `Sample_TerraClimate_2018.csv` on Line 17,367 - Line 20,865

In [3]:
# sample_file = '../data/Global_samples_Balanced_100.csv'
# sample_file = '../data/Global_Samples_Balanced_n100_m250_s210_attempt1.csv'
# sample_file = '../data/Global_Samples_Balanced_n100_m200_s211.csv'
sample_file = '../data/Global_Samples_Balanced_n200_m50_s213.csv'

In [4]:
world_df = pd.read_csv(sample_file, index_col=[0])
world_df.head()
world_df['geometry'] = world_df['geometry'].apply(wkt.loads)
world_df['samples'] = world_df['samples'].apply(literal_eval)
world_df['n_samples'] = world_df.apply(lambda row: len(row['samples']), axis=1)
world_gdf = geopandas.GeoDataFrame(world_df, geometry='geometry')
world_gdf = world_gdf[(world_gdf['n_samples']>0)]
world_gdf.head()

,country_code,geometry,area,samples,n_samples
1,FJI,"POLYGON ((178.12557 -17.50481, 178.37360 -17.3...",0.983740,"[[177.50428304875032, -17.681708374019223]]",1
3,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",41.629232,"[[35.660584639565975, -2.7239110740946204], [3...",6
4,TZA,"POLYGON ((39.20222 -4.67677, 38.74054 -5.90895...",34.672732,"[[37.56683623717293, -7.608781878929718], [37....",4
6,CAN,"POLYGON ((-70.19171 47.03804, -68.65000 48.300...",16.730885,"[[-63.59254938357725, 45.81992037628136], [-64...",6
7,CAN,"POLYGON ((-63.25471 44.67014, -64.24656 44.265...",2.471299,"[[-65.09463983153698, 44.94090517796168]]",1


In [5]:
country_lst = list(set(world_gdf['country_code'].tolist()))
country_lst.sort()
country_lst

['AFG',
 'AGO',
 'ALB',
 'ARG',
 'ARM',
 'AUS',
 'AUT',
 'AZE',
 'BEL',
 'BEN',
 'BFA',
 'BGD',
 'BGR',
 'BIH',
 'BLR',
 'BOL',
 'BRA',
 'BWA',
 'CAN',
 'CHE',
 'CHL',
 'CHN',
 'CIV',
 'CMR',
 'COD',
 'COG',
 'COL',
 'CUB',
 'CZE',
 'DEU',
 'DJI',
 'DNK',
 'DOM',
 'DZA',
 'ECU',
 'EGY',
 'ERI',
 'ESP',
 'EST',
 'ETH',
 'FIN',
 'FJI',
 'GAB',
 'GBR',
 'GEO',
 'GHA',
 'GIN',
 'GMB',
 'GNB',
 'GRC',
 'GUY',
 'HND',
 'HRV',
 'HTI',
 'HUN',
 'IDN',
 'IND',
 'IRL',
 'IRN',
 'IRQ',
 'ITA',
 'JPN',
 'KAZ',
 'KEN',
 'KGZ',
 'KHM',
 'KOR',
 'LBN',
 'LBR',
 'LBY',
 'LKA',
 'LSO',
 'LTU',
 'LVA',
 'MAR',
 'MDA',
 'MDG',
 'MEX',
 'MKD',
 'MLI',
 'MMR',
 'MNG',
 'MOZ',
 'MWI',
 'MYS',
 'NAM',
 'NCL',
 'NER',
 'NGA',
 'NIC',
 'NLD',
 'NPL',
 'NZL',
 'OMN',
 'PAK',
 'PAN',
 'PER',
 'PHL',
 'PNG',
 'POL',
 'PRK',
 'PRT',
 'PRY',
 'ROU',
 'RUS',
 'RWA',
 'SAU',
 'SDN',
 'SEN',
 'SOM',
 'SRB',
 'SSD',
 'SVK',
 'SWE',
 'SWZ',
 'SYR',
 'TCD',
 'TGO',
 'THA',
 'TJK',
 'TKM',
 'TLS',
 'TUN',
 'TUR',
 'TWN',


In [6]:
for i in country_lst:
# for i in rerun_lst:
    # Run on IBM Cloud
#     print('nohup python3 -u /git/Sample_TerraClimate_by_Country.py {} 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m250_s210_attempt1/TerraClimate_{}.log &'.format(i, i))
#     print('nohup python3 -u /git/Sample_TerraClimate_by_Country.py {} 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s210_RUS/TerraClimate_{}.log &'.format(i, i))
    print('nohup python3 -u /git/Sample_TerraClimate_by_Country.py {} 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s211/TerraClimate_{}.log &'.format(i, i))

nohup python3 -u /git/Sample_TerraClimate_by_Country.py AFG 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s211/TerraClimate_AFG.log &
nohup python3 -u /git/Sample_TerraClimate_by_Country.py AGO 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s211/TerraClimate_AGO.log &
nohup python3 -u /git/Sample_TerraClimate_by_Country.py ALB 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s211/TerraClimate_ALB.log &
nohup python3 -u /git/Sample_TerraClimate_by_Country.py ARG 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s211/TerraClimate_ARG.log &
nohup python3 -u /git/Sample_TerraClimate_by_Country.py ARM 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s211/TerraClimate_ARM.log &
nohup python3 -u /git/Sample_TerraClimate_by_Country.py AUS 2018 --resolution 30 > /git/GEE/TerraClimate_samples_n100_m200_s211/TerraClimate_AUS.log &
nohup python3 -u /git/Sample_TerraClimate_by_Country.py AUT 2018 --resolution 30 > /git/GEE/Te

#### Check the output from `Sample_Sentinel2_by_Country.py`

In [8]:
# output_dir = '../data/remote/TerraClimate_samples_n100_m250_s210_attempt1_noRUS'
# output_dir = '../data/remote/TerraClimate_samples_n100_m200_s210_RUS'
# output_dir = '../data/remote/TerraClimate_samples_n100_m200_s211'
output_dir = '../data/remote/TerraClimate_samples_n200_m50_s213'

* Check if GEE limit is reached when processing certain countries

In [9]:
error_msg = 'ee.ee_exception.EEException: Collection query aborted after accumulating over 5000 elements.'

In [10]:
# output_dir = '/git/GEE/TerraClimate_samples_n100_m250_s210_attempt1'
for f in os.listdir(output_dir):
    if f.endswith(".log"):
        with open(os.path.join(output_dir, f)) as ff:
            if error_msg in ff.read():
                print('===== {} ===='.format(f))
                print("True")

In [11]:
error_msg = 'ee.ee_exception.EEException: Quota exceeded for quota metric \'Number of read requests\' and limit \'Number of read requests per minute per user\' of service \'earthengine.googleapis.com\' for consumer'

In [12]:
rerun_lst = []
for f in os.listdir(output_dir):
    if f.endswith(".log"):
        with open(os.path.join(output_dir, f)) as ff:
            if error_msg in ff.read():
                print('===== {} ===='.format(f))
                print("True")
                rerun_lst.append(f[-7:-4])

===== TerraClimate_COL.log ====
True
===== TerraClimate_MMR.log ====
True


* Create a metadata for all the .csv file

In [13]:
csv_lst = [f.split('_') for f in os.listdir(output_dir) if f.endswith(".csv")]
csv_df = pd.DataFrame(csv_lst, columns=('country_code', 'segment', 'YYYYMM'))
# Remove .csv in YYYYMM
csv_df['YYYYMM'] = csv_df.apply(lambda row: row['YYYYMM'].split('.')[0], axis=1)
csv_df.head()

,country_code,segment,YYYYMM
0,AFG,0,201801
1,AFG,0,201802
2,AFG,0,201803
3,AFG,0,201804
4,AFG,0,201805


In [14]:
csv_df['country_segment_code'] = csv_df.apply(lambda row: row['country_code']+'_'+row['segment'], axis=1)
csv_df['value'] = 1
# YYYYMM is actually the column level names rather than index
# Reference: https://stackoverflow.com/questions/19851005/rename-pandas-dataframe-index
csv_pivot = csv_df.pivot(index='country_segment_code', columns='YYYYMM', values='value') \
                    .add_prefix('YM') \
                    .reset_index()
csv_pivot

YYYYMM,country_segment_code,YM201801,YM201802,YM201803,YM201804,YM201805,YM201806,YM201807,YM201808,YM201809,YM201810,YM201811,YM201812
0,AFG_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,AFG_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,AGO_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AGO_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,ALB_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,ARG_0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,ARG_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,ARG_2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,ARG_3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,ARG_4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Fine rows with NaN

In [15]:
is_NaN = csv_pivot.isnull()
row_has_NaN = is_NaN.any(axis=1)
csv_pivot[row_has_NaN]

YYYYMM,country_segment_code,YM201801,YM201802,YM201803,YM201804,YM201805,YM201806,YM201807,YM201808,YM201809,YM201810,YM201811,YM201812
84,CAN_18,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,COL_1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
249,MMR_1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Merge all sample data together

In [16]:
country_segment_lst = list(set(csv_df['country_segment_code'].tolist()))
country_segment_nan_lst = csv_pivot[row_has_NaN]['country_segment_code'].tolist()

TerraClimate_sample_df = pd.DataFrame()
for csc in country_segment_lst:
    if csc in country_segment_nan_lst:
        continue
    print(csc)
    country_sentinel2_lst = [f for f in os.listdir(output_dir) if f.endswith(".csv") and csc+'_' in f]
    if len(country_sentinel2_lst) == 0:
        continue
    # Create a dictionary of all csv pertaining to a country segment in pd.DataFrame
    country_sentinel2_df = dict()
    for f in country_sentinel2_lst:
        temp_df = pd.read_csv(os.path.join(output_dir, f))
        # Drop Unamed column
        temp_df = temp_df.loc[:, ~temp_df.columns.str.contains('^Unnamed')]
        country_sentinel2_df[f.split('.')[0]] = temp_df

    country_df_lst = list(country_sentinel2_df.values())
    # Merge using lat and lon
    country_merge_raw_df = reduce(lambda left, right: pd.merge(left,right,on=['lat', 'lon']), country_df_lst)
    
    country_merge_raw_df['country_segment_code'] = csc
    TerraClimate_sample_df = TerraClimate_sample_df.append(country_merge_raw_df, ignore_index=True, sort=True)

CAN_16
BRA_5
AUS_16
RUS_1
ARM_0
NGA_1
PHL_3
RUS_37
IRN_1
IRN_0
KAZ_5
BOL_1
SSD_0
IDN_4
THA_0
BRA_9
LBY_0
USA_10
GUY_0
ERI_0
USA_15
AZE_0
CHN_2
RUS_7
KAZ_2
USA_5
AUS_11
ARG_7
CHN_19
LSO_0
PER_0
BRA_15
DNK_1
BRA_1
RUS_20
CHN_29
BRA_18
RUS_46
MLI_0
MYS_1
USA_12
USA_25
PAK_1
RUS_50
USA_23
ARG_0
USA_7
USA_1
LVA_0
CHN_5
BRA_0
CAN_14
IND_6
RUS_4
IDN_3
BOL_2
JPN_2
PHL_1
KAZ_8
PRT_0
CHN_12
ZAF_0
BOL_0
CHN_16
PRK_0
GNB_0
HUN_0
BWA_0
DOM_0
RUS_0
USA_4
FIN_0
SDN_0
RUS_49
RUS_39
CAN_3
CAN_6
USA_24
AUS_17
IND_5
IND_8
IND_0
AUS_13
AUS_3
RUS_18
USA_2
KHM_0
RUS_48
IND_2
USA_31
RUS_25
IDN_0
MYS_0
ARG_6
LBR_0
TUR_1
HND_0
PHL_4
RUS_44
RUS_34
KAZ_9
RUS_33
AUS_7
PER_2
RUS_45
AUS_15
USA_0
MDG_0
TUR_3
BGR_0
RUS_43
AUS_6
MDG_1
SWZ_0
MWI_0
AGO_1
UKR_0
EST_0
BRA_13
USA_9
ETH_0
ETH_2
ESP_0
IRL_0
RUS_14
SRB_0
ARG_2
SAU_2
MLI_2
CHN_25
ITA_2
BLR_0
PAK_0
ROU_0
TCD_0
TUN_0
USA_33
COD_2
RWA_0
RUS_29
SAU_0
MEX_6
EGY_2
BRA_12
KEN_0
CHN_10
CHN_24
ZMB_0
AUS_2
IDN_7
CHN_7
PHL_2
COD_0
USA_36
PHL_6
NER_1
CHN_6
GEO_0
CAN_5
IDN

In [17]:
is_NaN = TerraClimate_sample_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
print(TerraClimate_sample_df[row_has_NaN].shape)
print(TerraClimate_sample_df.shape)

(18, 171)
(3499, 171)


In [18]:
# TerraClimate_sample_df.to_csv('../data/Sample_TerraClimate_2018_n200_m50_s213.csv')

In [72]:
TerraClimate_sample_df

,201801_aet,201801_def,201801_pdsi,201801_pet,201801_pr,201801_ro,201801_soil,201801_srad,201801_swe,201801_tmmn,...,201812_srad,201812_swe,201812_tmmn,201812_tmmx,201812_vap,201812_vpd,201812_vs,country_segment_code,lat,lon
0,0.0,0.0,200.0,0.0,2.0,0.0,100.0,6.0,41.0,-406.0,...,0.0,46.0,-385.0,-324.0,46.0,-2.0,270.0,CAN_23,79.478258,-86.659877
1,0.0,0.0,320.0,0.0,2.0,0.0,246.0,5.0,37.0,-378.0,...,0.0,44.0,-356.0,-293.0,40.0,0.0,310.0,CAN_23,79.327341,-87.897935
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAN_23,80.333903,-88.777296
3,780.0,0.0,220.0,780.0,120.0,42.0,1966.0,1543.0,0.0,161.0,...,1958.0,0.0,171.0,259.0,2202.0,61.0,150.0,HND_0,15.523420,-88.366586
4,611.0,329.0,150.0,940.0,26.0,1.0,1357.0,1848.0,0.0,118.0,...,2030.0,0.0,132.0,225.0,1607.0,55.0,270.0,HND_0,14.158430,-87.254921
5,670.0,684.0,-10.0,1354.0,4.0,0.0,1307.0,2091.0,0.0,158.0,...,2176.0,0.0,170.0,302.0,2044.0,113.0,210.0,HND_0,14.034732,-88.303524
6,889.0,0.0,110.0,889.0,256.0,167.0,2200.0,1551.0,0.0,199.0,...,1827.0,0.0,214.0,280.0,2550.0,57.0,200.0,HND_0,15.103817,-84.767486
7,786.0,107.0,250.0,893.0,63.0,3.0,1906.0,1614.0,0.0,193.0,...,2070.0,0.0,206.0,299.0,2498.0,97.0,110.0,HND_0,15.414274,-87.849965
8,732.0,854.0,-20.0,1586.0,5.0,0.0,1021.0,2166.0,0.0,180.0,...,2254.0,0.0,188.0,325.0,2208.0,144.0,270.0,HND_0,13.703523,-87.706594
9,687.0,730.0,-30.0,1417.0,8.0,0.0,1100.0,2132.0,0.0,161.0,...,2234.0,0.0,172.0,301.0,2039.0,116.0,280.0,HND_0,13.732898,-87.635447


In [73]:
# for ix in TerraClimate_sample_df.index: 
#     print(type(TerraClimate_sample_df.loc[ix]))

# TerraClimate_sample_df.loc[0]
feature_df = TerraClimate_sample_df.loc[:, TerraClimate_sample_df.columns.str.contains('2018')].dropna()

In [74]:
feature_df

,201801_aet,201801_def,201801_pdsi,201801_pet,201801_pr,201801_ro,201801_soil,201801_srad,201801_swe,201801_tmmn,...,201812_pr,201812_ro,201812_soil,201812_srad,201812_swe,201812_tmmn,201812_tmmx,201812_vap,201812_vpd,201812_vs
0,0.0,0.0,200.0,0.0,2.0,0.0,100.0,6.0,41.0,-406.0,...,5.0,0.0,100.0,0.0,46.0,-385.0,-324.0,46.0,-2.0,270.0
1,0.0,0.0,320.0,0.0,2.0,0.0,246.0,5.0,37.0,-378.0,...,6.0,0.0,246.0,0.0,44.0,-356.0,-293.0,40.0,0.0,310.0
3,780.0,0.0,220.0,780.0,120.0,42.0,1966.0,1543.0,0.0,161.0,...,88.0,4.0,1843.0,1958.0,0.0,171.0,259.0,2202.0,61.0,150.0
4,611.0,329.0,150.0,940.0,26.0,1.0,1357.0,1848.0,0.0,118.0,...,11.0,1.0,1320.0,2030.0,0.0,132.0,225.0,1607.0,55.0,270.0
5,670.0,684.0,-10.0,1354.0,4.0,0.0,1307.0,2091.0,0.0,158.0,...,5.0,0.0,1716.0,2176.0,0.0,170.0,302.0,2044.0,113.0,210.0
6,889.0,0.0,110.0,889.0,256.0,167.0,2200.0,1551.0,0.0,199.0,...,148.0,50.0,2200.0,1827.0,0.0,214.0,280.0,2550.0,57.0,200.0
7,786.0,107.0,250.0,893.0,63.0,3.0,1906.0,1614.0,0.0,193.0,...,45.0,2.0,1279.0,2070.0,0.0,206.0,299.0,2498.0,97.0,110.0
9,687.0,730.0,-30.0,1417.0,8.0,0.0,1100.0,2132.0,0.0,161.0,...,4.0,0.0,1476.0,2234.0,0.0,172.0,301.0,2039.0,116.0,280.0
10,759.0,75.0,190.0,834.0,66.0,3.0,1774.0,1646.0,0.0,156.0,...,70.0,4.0,1645.0,2000.0,0.0,169.0,262.0,2155.0,65.0,140.0
11,442.0,0.0,-160.0,442.0,161.0,30.0,2081.0,997.0,0.0,36.0,...,214.0,40.0,2081.0,897.0,0.0,44.0,114.0,782.0,35.0,250.0


In [43]:
# Input tensor to RNN
# [
#     # Timestep 1
#     [ temperature_in_paris, value_of_nasdaq, unemployment_rate ],
#     # Timestep 2
#     [ temperature_in_paris, value_of_nasdaq, unemployment_rate ],
#     # Timestep 3
#     [ temperature_in_paris, value_of_nasdaq, unemployment_rate ],
#     ...
# ]
# reshape(observations, timestamp, features)
feature_df.values.reshape(4239, 12, 14)

array([[[   0.,    0.,  200., ...,   49.,   -3.,  240.],
        [   0.,    0.,  160., ...,   45.,   -2.,  240.],
        [   0.,    0.,  140., ...,   45.,   -2.,  190.],
        ...,
        [   0.,    0.,  310., ...,  125.,   -2.,  250.],
        [   0.,    0.,  280., ...,   56.,   -1.,  200.],
        [   0.,    0.,  280., ...,   46.,   -2.,  270.]],

       [[   0.,    0.,  320., ...,   39.,   -1.,  290.],
        [   0.,    0.,  280., ...,   39.,   -1.,  270.],
        [   0.,    0.,  250., ...,   37.,   -1.,  220.],
        ...,
        [   0.,    0.,  560., ...,  133.,    1.,  280.],
        [   0.,    0.,  510., ...,   56.,    0.,  230.],
        [   0.,    0.,  490., ...,   40.,    0.,  310.]],

       [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [  nan,   nan,   nan, ...,   nan,   nan,   nan],
        ...,
        [  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [  nan,   nan,   nan, ...,   nan,   n

In [39]:
feature_df.columns

Index(['201801_aet', '201801_def', '201801_pdsi', '201801_pet', '201801_pr',
       '201801_ro', '201801_soil', '201801_srad', '201801_swe', '201801_tmmn',
       ...
       '201812_pr', '201812_ro', '201812_soil', '201812_srad', '201812_swe',
       '201812_tmmn', '201812_tmmx', '201812_vap', '201812_vpd', '201812_vs'],
      dtype='object', length=168)

In [40]:
TerraClimate_sample_df.columns

Index(['201801_aet', '201801_def', '201801_pdsi', '201801_pet', '201801_pr',
       '201801_ro', '201801_soil', '201801_srad', '201801_swe', '201801_tmmn',
       ...
       '201812_srad', '201812_swe', '201812_tmmn', '201812_tmmx', '201812_vap',
       '201812_vpd', '201812_vs', 'country_segment_code', 'lat', 'lon'],
      dtype='object', length=171)